In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Support for FTrace events parsing and visualization
import trappy

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Step, Pulse, Periodic

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Let's use the local host as a target
te = TestEnv(
    # Setup a target configuration
    target_conf = {
    
    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # Linux system, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'juno2',   # load JUNO specific settings, e.g.
                              # - HWMON based energy sampling
                              # - Juno energy model
                              # valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - oak   - Mediatek MT63xx based target

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
             "sched_switch",
             "cpu_frequency"
         ],
         "buffsize" : 10240
    },

    # Account to access the remote target
    "host"        : '10.1.210.36',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
    #"rtapp-calib" : {
    #    '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
    #}
    "rtapp-calib" : { "0": 323, "1": 124, "2": 125, "3": 319, "4": 315, "5": 315 }

}

)

10:17:20  INFO    :         Target - Using base path: /data/work/juno/lisa
10:17:20  INFO    :         Target - Loading custom (inline) target configuration
10:17:20  DEBUG   :         Target - Target configuration {'username': 'root', 'platform': 'android', 'host': '10.1.210.36', 'ftrace': {'buffsize': 10240, 'events': ['sched_switch', 'cpu_frequency']}, 'board': 'juno2', 'rtapp-calib': {'1': 124, '0': 323, '3': 319, '2': 125, '5': 315, '4': 315}, 'password': '', 'tools': ['rt-app', 'taskset', 'trace-cmd']}
10:17:20  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
10:17:20  INFO    :         Target - Connecting Android target [10.1.210.36:5555]
10:17:20  INFO    :         Target - Connection settings:
10:17:20  INFO    :         Target -    {'device': '10.1.210.36:5555'}
10:17:20  DEBUG   :         Target - Setup ANDROID target...
10:17:20  DEBUG   : Installing module vexpress-u-boot
10:17:20  DEBUG   : Installing module vexpress-dtr
10:17:20  DEBUG   : I

# Create a new RTA workload generator object

The wlgen::RTA class is a workload generator which exposes an API to configure
RTApp based workload as well as to execute them on a target.

In [4]:
# Create a new RTApp workload generator
rtapp = RTA(
    
    target=te.target, # Target execution on the local machine
    
    name='example', # This is the name of the JSON configuration file reporting
                    # the generated RTApp configuration
    
    #calibration={0: 10, 1: 11, 2: 12, 3: 13} # These are a set of fake
    #                                         # calibration values
)

10:17:24  INFO    :          WlGen - Setup new workload example
10:17:24  DEBUG   :          WlGen - Setup step [postrun] callback to [__postrun] function


In [5]:
# Initial phase and pinning parameters
#ramp = Ramp(period_ms=100, start_pct=5, end_pct=65, delta_pct=20, time_s=1,
#            cpus="0")

# Following phases
#medium_slow = Periodic(duty_cycle_pct=10, duration_s=5, period_ms=100)
#high_fast   = Periodic(duty_cycle_pct=60, duration_s=5, period_ms=10)
#medium_fast = Periodic(duty_cycle_pct=10, duration_s=5, period_ms=1)
#high_slow   = Periodic(duty_cycle_pct=60, duration_s=5, period_ms=100)

#Compose the task
#complex_task = ramp + medium_slow + high_fast + medium_fast + high_slow


In [7]:
def build_perf_benchmark_rtapp(run_duration_ms, calibration_cpu_name, num_tasks, iterations, logdir="/data/local/tmp", file_name="perfbench.json"):
    # static content
    json_content = { 
        'global': {
            'calibration': calibration_cpu_name,
            'default_policy': "SCHED_OTHER",
            'duration': -1,
            'logdir': logdir
        },
        'tasks': {
           'controller': {
                'loop': iterations+2, 
                'phases': {
                    'init_delay': {
                        'sleep': run_duration_ms*4
                    }
                }
            }
        }
    }
    # dynamic content (number of tasks)
    for cpu in range(0,num_tasks):
        bench_thread_name = "bench{}".format(cpu)
        # describe the worker thread
        json_content['tasks'][bench_thread_name] = {
                'loop': iterations, 
                'phases': {
                    'go': {
                        'run': run_duration_ms
                    }, 
                    'wait': {
                        'suspend': bench_thread_name
                    }
                }
            }
        # hook it to the controller
        json_content['tasks']['controller']['phases']["trigger{}".format(cpu)] = { 'resume': bench_thread_name }
    
    with open(file_name, 'w') as outfile:
            json.dump(json_content, outfile,
                    sort_keys=True, indent=4, separators=(',', ': '))
    return (file_name, json_content)
    

In [8]:
run_duration_ms = 500000
calibration = 'CPU1'
num_tasks = 6
iterations = 8

(filename, json_data)=build_perf_benchmark_rtapp(run_duration_ms, calibration, num_tasks, iterations)

# Configure this RTApp instance to:
name=rtapp.conf(
    # 1. generate a "profile based" set of tasks
    #kind='profile',
    kind='custom',
    duration=-1,    
    # 2. define the "profile" of each task
    params={
        # use filename
#        'custom_filename' : "{}".format(filename)
        # use variable directly
        'custom_config': json_data
    },

    # 6. use this folder for task logfiles
    run_dir='/data/local/tmp'
)

with open('{}'.format(rtapp.json), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)


10:17:34  DEBUG   :          WlGen - Configuring custom workload...
10:17:34  DEBUG   :          RTApp - ref on cpu: 1
10:17:34  DEBUG   : adb -s 10.1.210.36:5555 push 'example_00.json' '/data/local/tmp'
10:17:34  INFO    : Generated RTApp JSON file:


tmp_json = 
{
    "global": {
        "calibration": "CPU1",
        "default_policy": "SCHED_OTHER",
        "duration": -1,
        "logdir": "/data/local/tmp"
    },
    "tasks": {
        "bench0": {
            "loop": 8,
            "phases": {
                "go": {
                    "run": 500000
                },
                "wait": {
                    "suspend": "bench0"
                }
            }
        },
        "bench1": {
            "loop": 8,
            "phases": {
                "go": {
                    "run": 500000
                },
                "wait": {
                    "suspend": "bench1"
                }
            }
        },
        "bench2": {
            "loop": 8,
            "phases": {
                "go": {
                    "run": 500000
                },
                "wait": {
                    "suspend": "bench2"
                }
            }
        },
        "bench3": {
            "loop": 8,
            "p

In [9]:


def configure_target(is_big_little=0, initial_task_util=0, num_of_cpus=6, governor="sched", disabled_idle_states=[]):
    # device paths
    isBigLittle='/proc/sys/kernel/sched_is_big_little'
    initialTaskUtil='/proc/sys/kernel/sched_initial_task_util'
    cpuFreqGovernor='/sys/devices/system/cpu/cpu{}/cpufreq/scaling_governor'
    cpuIdleState='/sys/devices/system/cpu/cpu{}/cpuidle/state{}/disable'
    
    # Configure 'is_big_little' flag
    te.target.execute("echo {} > {}".format(is_big_little, isBigLittle))
    # Configure 'initial_task_util' flag
    te.target.execute("echo {} > {}".format(initial_task_util, initialTaskUtil))
    # set cpufreq governors
    te.target.execute("echo {} > {}".format(governor, cpuFreqGovernor.format(0)))
    te.target.execute("echo {} > {}".format(governor, cpuFreqGovernor.format(1)))
    # configure idle states
    idle_states=[ 0,1,2 ]
    if len(disabled_idle_states):
        for state in disabled_idle_states:
            idle_states.remove(state)
    
    for cpu in range(0, num_of_cpus):
        # disable states
        if len(disabled_idle_states):
            for idle_state in disabled_idle_states:
                te.target.execute("echo 0 > {}".format(cpuIdleState.format(cpu, idle_state)))
        #enable states
        if len(idle_states):
            for idle_state in idle_states:
                te.target.execute("echo 1 > {}".format(cpuIdleState.format(cpu, idle_state)))
    # report
    print "sched_is_big_little:     {}".format(te.target.read_value(isBigLittle))
    print "sched_initial_task_util: {}".format(te.target.read_value(initialTaskUtil))
    print "Little Cluster Governor: {}".format(te.target.read_value(cpuFreqGovernor.format(0)))
    print "Big Cluster Governor:    {}".format(te.target.read_value(cpuFreqGovernor.format(1)))
    idle_states=[ 0,1,2 ]
    for cpu in range(0, num_of_cpus):
        print "Cpu {} Idle States:".format(cpu)
        for state in idle_states:
            val = te.target.read_value(cpuIdleState.format(cpu, state))
            result='disabled'
            if val > 0:
                result='enabled'
            print "  State {} : {}".format(state, result)

In [10]:
configure_target(governor='sched')

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 0 > /proc/sys/kernel/sched_is_big_little"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 0 > /proc/sys/kernel/sched_initial_task_util"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo sched > /sys/devices/system/cpu/cpu0/cpufreq/scaling_governor"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo sched > /sys/devices/system/cpu/cpu1/cpufreq/scaling_governor"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 1 > /sys/devices/system/cpu/cpu0/cpuidle/state0/disable"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 1 > /sys/devices/system/cpu/cpu0/cpuidle/state1/disable"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 1 > /sys/devices/system/cpu/cpu0/cpuidle/state2/disable"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 1 > /sys/devices/system/cpu/cpu1/cpuidle/state0/disable"
10:18:20  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 1 > /sys/devices/system/cpu/cpu1/cpuidle/state1/disabl

sched_is_big_little:     0
sched_initial_task_util: 0
Little Cluster Governor: sched
Big Cluster Governor:    sched
Cpu 0 Idle States:
  State 0 : enabled
  State 1 : enabled
  State 2 : enabled
Cpu 1 Idle States:
  State 0 : enabled
  State 1 : enabled


10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu2/cpuidle/state0/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu2/cpuidle/state1/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu2/cpuidle/state2/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu3/cpuidle/state0/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu3/cpuidle/state1/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu3/cpuidle/state2/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu4/cpuidle/state0/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu4/cpuidle/state1/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu4/cpuidle/state2/disable'"


  State 2 : enabled
Cpu 2 Idle States:
  State 0 : enabled
  State 1 : enabled
  State 2 : enabled
Cpu 3 Idle States:
  State 0 : enabled
  State 1 : enabled
  State 2 : enabled
Cpu 4 Idle States:
  State 0 : enabled
  State 1 : enabled


10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu5/cpuidle/state0/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu5/cpuidle/state1/disable'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/devices/system/cpu/cpu5/cpuidle/state2/disable'"
10:18:21  INFO    : #### Setup FTrace
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 'echo 10240 > '\\''/sys/kernel/debug/tracing/buffer_size_kb'\\''' | su"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/kernel/debug/tracing/buffer_size_kb'"
10:18:21  DEBUG   : adb -s 10.1.210.36:5555 shell "echo '/data/local/tmp/bin/trace-cmd reset' | su"


  State 2 : enabled
Cpu 5 Idle States:
  State 0 : enabled
  State 1 : enabled
  State 2 : enabled


10:18:23  DEBUG   : adb -s 10.1.210.36:5555 shell "echo '/data/local/tmp/bin/trace-cmd start -e sched_switch -e cpu_frequency' | su"
10:18:25  DEBUG   : adb -s 10.1.210.36:5555 shell "echo 'echo TRACE_MARKER_START > '\\''/sys/kernel/debug/tracing/trace_marker'\\''' | su"
10:18:25  DEBUG   : Trace CPUFreq frequencies
10:18:25  DEBUG   : adb -s 10.1.210.36:5555 shell "echo '/data/local/tmp/bin/shutils cpufreq_trace_all_frequencies' | su"
10:18:25  INFO    : #### Start energy sampling
10:18:25  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/class/hwmon/hwmon0/energy2_input'"
10:18:25  DEBUG   : adb -s 10.1.210.36:5555 shell "cat '/sys/class/hwmon/hwmon0/energy3_input'"
10:18:25  DEBUG   : SAMPLE: {'BOARDBIG': {'total': 0, 'last': 13396.05908, 'delta': 0}, 'BOARDLITTLE': {'total': 0, 'last': 10221.74299, 'delta': 0}}
10:18:25  DEBUG   : RESET: {'BOARDBIG': {'total': 0, 'last': 13396.05908, 'delta': 0}, 'BOARDLITTLE': {'total': 0, 'last': 10221.74299, 'delta': 0}}
10:18:25  INFO    : ##

In [ ]:
configure_target(1,0)

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [ ]:
configure_target(0,1024)

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [ ]:
configure_target(1,1024)

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)